In [1]:
%matplotlib inline 


This notebook deals with banks of cylinders in a cross flow. Cylinder banks are common heat exchangers where the cylinders may be heated by electricity or a fluid may be flowing within the cylinder to cool or heat the flow around the cylinders. The advantage of cylinder banks is the increase mixing in the fluid, thus the temperature downstream of the bank is likely to be quite homogeneous.

<img src='figures_Tube_Banks/fig_07_11.jpg' alt="my awesome sketch" width=50% >

The arrangement of cylinders may be aligned or staggered as shown in the following figures. The flow and geometrical parameters will be used in the derivation of temperature equations and Nusselt number correlation.
<img src='figures_Tube_Banks/fig_07_12.jpg' alt="my awesome sketch" width=50% >

This notebook should cover a wide variety of problems, providing that the assumption of isothermal boundary conditions on the tubes is (approximately) valid. The tube surface temperature is $T_s$. 
The flow and geometrical parameters of importance to solve this problem are:

* Arithmetic mean of temperature between inlet $T_i$ and outlet $T_o$ of the bank. 
$$
T_m = \frac{T_i+T_o}{2}
$$
* Reynolds number based on the max velocity within the bank $V_\text{max}$, the density and viscosity based on $T_m$:
$$
Re=\frac{\rho V_\text{max}D}{\mu}
$$
**Question: At what temperature should you estimate $\rho$ and $\mu$?** The energy of the flow comes from the inlet and the velocity $V_\mathrm{max}$ is calculated from the inlet velocity. The density should therefore be estimated at $T_i$. The viscous forces however occur throughout the domain, so $\mu$ should be estimated at $T_m$. In some cases $T_o$ is the quantity to be found. It is acceptable to use $\mu(T_i)$, but you must verify that the temperature difference $\Delta T=\vert T_i-T_o\vert$ is not too large. If it is, you must repeat the calculation iteratively with  $\mu(T_i)$ until $T_o$ converges.

* Prandtl number $Pr$ based on $T_m$ 
* Surface Prandtl number $Pr_s$ based on $T_s$
* Number of tubes in the transversal direction $N_T$, longitudinal direction $N_L$ and total $N=N_T\times N_L$
* The transversal $S_T$ and longitudinal $S_L$ separations between tubes in a row and between rows.
* The type of tube arrangement: 
    * Aligned
$$
V_\text{max}=\frac{S_T}{S_T-D}V_i
$$
    * Staggered
$$
V_\text{max}=\frac{S_T}{2(S_D-D)}V_i\text{ with }S_D=\sqrt{S_L^2+\left(\frac{S_T}{2}\right)^2}
$$


The Nusselt number correlation for a bank of tubes is a variation of the Zukauskas correlation:
$$
Nu = C_2C_1Re^mPr^{0.36}\left(\frac{Pr}{Pr_s}\right)^{1/4}
$$
where $C_2$ depends on $N_L$. In the library, the function for this correlation is
<FONT FACE="courier" style="color:red">Nu_tube_banks(Re,Pr,Pr_s,S_L,S_T,N_L,arrangement) </FONT>.

The heat rate per unit length across the tube bank is
$$
q'=N\overline{h}\pi D \Delta T_\text{lm}
$$
where the temperature drop is the log-mean temperature difference
$$
\Delta T_\text{lm}=\cfrac{(T_s-T_i)-(T_s-T_o)}{\ln\left(\cfrac{T_s-T_i}{T_s-T_o}\right)}
$$
which accounts for the exponential variation of temperature across the bank
$$
\cfrac{T_s-T_o}{T_s-T_i}=\exp\left(-\cfrac{\pi D N \overline{h}}{\rho V_i N_T S_T C_p}\right)
$$
where $\rho$, $C_p$ and $V_i$ are inlet quantities if $T_o$ is unknown of the arthimetic mean temperature if available. Note that $N=N_L\times N_T$ thus 
$$
\cfrac{T_s-T_o}{T_s-T_i}=\exp\left(-\cfrac{\pi D N_L \overline{h}}{\rho V_i S_T C_p}\right)
$$
One may want to determine the number of tubes necessary to achieve a given $T_o$. The number of tubes in the transverse directions is typically dictated by the geometry of the system, so we are looking for $N_L$:
$$
N_L = \cfrac{\rho V_i S_T C_p}{\pi D \overline{h}} \log\left(\cfrac{T_s-T_i}{T_s-T_o}\right)
$$


The pressure loss through the tube bank is a critical component of the heat exchanger design. The presence of obstacles in the flow requires an increase in the mechanical energy necessary to drive the flow at a given flow rate. The pressure loss, given all parameters above, is
$$
\Delta p = N_L\,\chi\, f\,\frac{\rho V_\text{max}^2}{2}
$$
where the friction factor $f$ and the parameter $\chi$ are given by the graphs below for the aligned (top) and staggered (bottom) arrangements. These graphs use two new quantities, the longitudnal and transverse pitches:
$$
P_L=\frac{S_L}{D}\text{ and } P_T=\frac{S_T}{D}
$$
<img src='figures_Tube_Banks/fig_07_14.jpg' alt="my awesome sketch" width=100% >
<img src='figures_Tube_Banks/fig_07_15.jpg' alt="my awesome sketch" width=100% >

## Problem1
A preheater involves the use of condensing steam on the inside of a bank of tubes to heat air that enters at $P_i=1 \text{ atm}$ and $T_i=25^\circ\text{C}$. The air moves at $V_i=5\text{ m/s}$ in cross flow over the tubes. Each tube is $L=1\text{ m}$ long and has an outside diameter of $D=10 \text{ mm}$. The bank consists of columns of 14 tubes in the transversal direction $N_T=14$ and $N_L$ rows in the direction of flow. The arrangement of tubes is aligned array for which $S_T=S_L=15\text{ mm}$. What is the minimum value of $N_L$ needed to achieve an outlet temperature of $T_o=75^\circ\text{C}$? What is the corresponding pressure drop across the tube bank?


In [12]:
import numpy as np
from Libraries import thermodynamics as thermo
from Libraries import HT_external_convection as extconv

T_i = 25 #C
T_o = 75 #C
T_s = 100 #C
V_i = 5 #m/s
L = 1 #m
D = 10e-3 #mm
N_L = 14
S_T = S_L = 15e-3 #m

bank = extconv.BankofTubes('aligned','air',T_i,T_s,T_o,"C",V_i,D,S_L,S_T,N_L)
print(bank.N_L_for_given_To)


15.25542247062167


## Problem 2

A preheater involves the use of condensing steam at $100^\circ\text{C}$ on the inside of a bank of tubes to heat air that enters at $1 \text{ atm}$ and $25^\circ\text{C}$. The air moves at $5\text{ m/s}$ in cross flow over the tubes. Each tube is $1\text{ m}$ long and has an outside diameter of $10 \text{ mm}$. The bank consists of 196 tubes in a square, aligned array for which $S_T=S_L=15\text{ mm}$. What is the total rate of heat transfer to the air? What is the pressure drop associated with the airflow?

## Problem 3

<img src='figures_Tube_Banks/probun_07_34.jpg' alt="my awesome sketch" width=100% >
An air duct heater consists of an aligned array of electrical heating elements in which the longitudinal and transverse pitches are $S_L=S_T= 24\text{ mm}$. There are 3 rows of elements in the ﬂow direction ($N_L=3$) and 4 elements per row ($N_T=4$). Atmospheric air with an upstream velocity of $12\text{ m/s}$ and a temperature of $25^\circ\text{C}$ moves in cross ﬂow over the elements, which have a diameter of $12\text{ mm}$, a length of $250\text{ mm}$, and are maintained at a surface temperature of $350^\circ\text{C}$.
<ol>
<li>
Determine the total heat transfer to the air and the temperature of the air leaving the duct heater.
</li>
<li>
Determine the pressure drop across the element bank and the fan power requirement.
</li>
<li>
Compare the average convection coefﬁcient obtained in your analysis with the value for an isolated (single) element. Explain the difference between the results.
</li>
<li>
What effect would increasing the longitudinal and transverse pitches to 30 mm have on the exit temperature of the air, the total heat rate, and the pressure drop?
</li>
</ol>